In [4]:
import getpass

from snowflake.snowpark import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F

In [6]:
pwd = getpass.getpass('Enter ACCOUNTADMIN password:  ')

Enter ACCOUNTADMIN password:   ········


In [12]:
params = dict(
    account='wd03530.us-central1.gcp',
    user='mtulow',
    password=pwd,
    database='nyc_taxi',
    schema='public',
    warehouse='compute_wh',
)

session = Session.builder.configs(params).create()

In [14]:
session.sql('ls @stage/yellow').show()

session.sql('ls @stage/green').show()

session.sql('ls @stage/fhv').show()

------------------------------------------------------------------------------------------------------------------------------------
|"name"                                             |"size"     |"md5"                             |"last_modified"                |
------------------------------------------------------------------------------------------------------------------------------------
|stage/yellow/2019/yellow_tripdata_2019-01.parquet  |110439648  |780f13de404692b0af539742fb667ee3  |Sat, 24 Dec 2022 07:48:53 GMT  |
|stage/yellow/2019/yellow_tripdata_2019-02.parquet  |103356032  |7264036b9a3a72a2a1508ce774de7f69  |Sat, 24 Dec 2022 07:49:04 GMT  |
|stage/yellow/2019/yellow_tripdata_2019-03.parquet  |116017376  |b69b3b7cf4ec4578161ff3e525e263d6  |Sat, 24 Dec 2022 07:49:11 GMT  |
|stage/yellow/2019/yellow_tripdata_2019-04.parquet  |110139152  |dec8bcf0f0b1a2399c7f74b1cfb2fae7  |Sat, 24 Dec 2022 07:49:18 GMT  |
|stage/yellow/2019/yellow_tripdata_2019-05.parquet  |111478944  |b4fe

In [ ]:
T.StructType([
    T.StructField('vendorid', T.IntegerType()),
    T.StructField('ratecodeid', T.IntegerType()),
    T.StructField('pickup_locationid', T.IntegerType()),
    T.StructField('dropoff_locationid', T.IntegerType()),
])

In [ ]:
session.sql('create or replace table public.yellow_tripdata ('
            # identifiers
            '    $1:vendorid::integer as vendorid,'
            '    $1:ratecodeid::integer as ratecodeid,'
            '    $1:pulocationid::integer as pickup_locationid,'
            '    $1:dolocationid::integer as dropoff_locationid,'
            # timestamps
            '    $1:tpep_pickup_datetime::timestamp as pickup_datetime,'
            '    $1:tpep_dropoff_datetime::timestamp as dropoff_datetime,'
            # trip into
            '    $1:fare_amount::numeric as fare_amount,'
            '    $1:extra::numeric as extra,'
            '    $1:mta_tax::numeric as mta_tax,'
            '    $1:tip_amount::numeric as tip_amount,'
            '    $1:tolls_amount as numeric as tolls_amount,'
            '    0 as ehail_fee,'
            '    $1:improvement_surcharge::numeric as improvement_surcharge,'
            '    $1:total_amount::numeric as total_amount,'
            '    $1:payment_type::integer as payment_type,'

    -- trip info
    store_and_fwd_flag,
    cast(passenger_count as integer) as passenger_count,
    cast(trip_distance as numeric) as trip_distance,
    -- yellow cabs are always street-hail
    1 as trip_type,
    
    -- payment info
    cast(fare_amount as numeric) as fare_amount,
    cast(extra as numeric) as extra,
    cast(mta_tax as numeric) as mta_tax,
    cast(tip_amount as numeric) as tip_amount,
    cast(tolls_amount as numeric) as tolls_amount,
    cast(0 as numeric) as ehail_fee,
    cast(improvement_surcharge as numeric) as improvement_surcharge,
    cast(total_amount as numeric) as total_amount,
    cast(payment_type as integer) as payment_type,
    {{ get_payment_type_description('payment_type') }} as payment_type_description, 
    cast(congestion_surcharge as numeric) as congestion_surcharge